# Books
**Author**: [Jake Rood](https://github.com/jakerood)

**Date**: 09-20-2023

## Setup

In [1]:
# Import the sqlite3 module
import sqlite3

# Set up a connection to books.db using sqlite3's connect function
connection = sqlite3.connect('books.db')

## Tables

In [2]:
# View the database's tables
# We need to import pandas first
import pandas as pd

# Set the max columns displayed to 10
pd.options.display.max_columns = 10

# Use pandas read_sql function to select everything from the authors table
pd.read_sql('SELECT * FROM authors', connection, index_col=['id'])

,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel
4,Dan,Quirk
5,Alexander,Wald


In [3]:
# Do the same for the titles table
pd.read_sql('SELECT * FROM titles', connection)

,isbn,title,edition,copyright
0,0135404673,Intro to Python for CS and DS,1,2020
1,0132151006,Internet & WWW How to Program,5,2012
2,0134743350,Java How to Program,11,2018
3,0133976890,C How to Program,8,2016
4,0133406954,Visual Basic 2012 How to Program,6,2014
5,0134601548,Visual C# How to Program,6,2017
6,0136151574,Visual C++ How to Program,2,2008
7,0134448235,C++ How to Program,10,2017
8,0134444302,Android How to Program,3,2017
9,0134289366,Android 6 for Programmers,3,2016


In [4]:
# Now do the same for the author_ISBN table but view just the first five rows
df = pd.read_sql('SELECT * FROM author_ISBN', connection)
df.head()

,id,isbn
0,1,0134289366
1,2,0134289366
2,5,0134289366
3,1,0135404673
4,2,0135404673


## 1 - SELECT

In [5]:
# Use SELECT to retrieve only the columns "first" and "last" from the authors table
pd.read_sql('SELECT first, last FROM authors', connection)

,first,last
0,Paul,Deitel
1,Harvey,Deitel
2,Abbey,Deitel
3,Dan,Quirk
4,Alexander,Wald


## 2 - WHERE

In [6]:
# Use SELECT and WHERE to view the title, edition and copyright for all books with copyright years greater than 2016
pd.read_sql("""SELECT title, edition, copyright FROM titles WHERE copyright > '2016'""", connection)

,title,edition,copyright
0,Intro to Python for CS and DS,1,2020
1,Java How to Program,11,2018
2,Visual C# How to Program,6,2017
3,C++ How to Program,10,2017
4,Android How to Program,3,2017


In [7]:
# Now locate all authors whose last name starts with the letter 'D'
pd.read_sql("""SELECT id, first, last FROM authors WHERE last LIKE 'D%'""", connection, index_col=['id'])

,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel


In [8]:
# Select the rows of all authors whos first names start with any character, followed by 'b', followed by

# any number of additional characters
pd.read_sql("""SELECT id, first, last FROM authors WHERE first LIKE '_b%'""", connection, index_col=['id'])

,first,last
id,,
3,Abbey,Deitel


## 3 - ORDER BY

In [9]:
# Use ORDER BY to sort the titles in ascending order
pd.read_sql('SELECT title FROM titles ORDER BY title ASC', connection)

,title
0,Android 6 for Programmers
1,Android How to Program
2,C How to Program
3,C++ How to Program
4,Internet & WWW How to Program
5,Intro to Python for CS and DS
6,Java How to Program
7,Visual Basic 2012 How to Program
8,Visual C# How to Program
9,Visual C++ How to Program


In [10]:
# Sort the authprs' names by last names, then by first name for any authors who have the same last name
pd.read_sql("""SELECT id, first, last FROM authors ORDER BY last, first""", connection, index_col=['id'])

,first,last
id,,
3,Abbey,Deitel
2,Harvey,Deitel
1,Paul,Deitel
4,Dan,Quirk
5,Alexander,Wald


In [11]:
# Sort the authors in descending order by last name and ascending order by first name for any authors

# who have the same last name
pd.read_sql("""SELECT id, first, last FROM authors ORDER BY last DESC, first ASC""", connection, index_col=['id'])

,first,last
id,,
5,Alexander,Wald
4,Dan,Quirk
3,Abbey,Deitel
2,Harvey,Deitel
1,Paul,Deitel


In [13]:
# Combine WHERE and ORDER BY to ge the isbn, title, edition and copyright of each book in the

# titles tables that has a title ending with 'How to Program' and sort them in ascending order by title
pd.read_sql("""SELECT isbn, title, edition, copyright FROM titles WHERE title LIKE '%How to Program'
                ORDER BY title""", connection)

,isbn,title,edition,copyright
0,0134444302,Android How to Program,3,2017
1,0133976890,C How to Program,8,2016
2,0134448235,C++ How to Program,10,2017
3,0132151006,Internet & WWW How to Program,5,2012
4,0134743350,Java How to Program,11,2018
5,0133406954,Visual Basic 2012 How to Program,6,2014
6,0134601548,Visual C# How to Program,6,2017
7,0136151574,Visual C++ How to Program,2,2008


## 4 - INNER JOIN

In [14]:
# Produce a list of authors accompanied by the ISBNs for books written by each author

# There are many entries, so just show the head of the result
pd.read_sql("""SELECT first, last, isbn FROM authors INNER JOIN author_ISBN
                ON authors.id = author_ISBN.id ORDER BY last, first""", connection).head()

,first,last,isbn
0,Abbey,Deitel,0132151006
1,Abbey,Deitel,0133406954
2,Harvey,Deitel,0134289366
3,Harvey,Deitel,0135404673
4,Harvey,Deitel,0132151006


## 5 - INSERT INTO

In [15]:
# To modify the database, we need to first obtain a sqlite3 Cursor object
cursor = connection.cursor()

# Use INSERT INTO to insert a new author named Sue Red into the authors table
cursor = cursor.execute("""INSERT INTO authors (first, last) VALUES ('Sue', 'Red')""")

In [16]:
# Query the authors table contents to view the newly added row
pd.read_sql('SELECT * FROM authors', connection, index_col=['id'])

,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel
4,Dan,Quirk
5,Alexander,Wald
6,Sue,Red


## 6 - UPDATE

In [17]:
# Update Sue Red's last name to 'Black'
cursor = cursor.execute("""UPDATE authors SET last='Black'
                            WHERE last='Red' AND first='Sue'""")

In [18]:
# Use the Cursor object's rowcount attribute to confirm the previous UPDATE modified one row
cursor.rowcount

1

In [19]:
# The UPDATE can also be confirmed by viewing the contents of the authors table
pd.read_sql('SELECT * FROM authors', connection, index_col=['id'])

,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel
4,Dan,Quirk
5,Alexander,Wald
6,Sue,Black


## 7 - DELETE FROM

In [20]:
# Use DELETE FROM to remove Sue Black from the authors table using her author ID
cursor = cursor.execute('DELETE FROM authors WHERE id=6')
cursor.rowcount

1

In [21]:
# View the authors table after Sue Black has been removed
pd.read_sql('SELECT * FROM authors', connection, index_col=['id'])

,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel
4,Dan,Quirk
5,Alexander,Wald


## 8 - SELECT

### authors

In [22]:
# View the final state of the authors table
pd.read_sql('SELECT * FROM authors', connection, index_col=['id'])

,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel
4,Dan,Quirk
5,Alexander,Wald


### titles

In [23]:
# View the final state of the titles table
pd.read_sql('SELECT * FROM titles', connection)

,isbn,title,edition,copyright
0,0135404673,Intro to Python for CS and DS,1,2020
1,0132151006,Internet & WWW How to Program,5,2012
2,0134743350,Java How to Program,11,2018
3,0133976890,C How to Program,8,2016
4,0133406954,Visual Basic 2012 How to Program,6,2014
5,0134601548,Visual C# How to Program,6,2017
6,0136151574,Visual C++ How to Program,2,2008
7,0134448235,C++ How to Program,10,2017
8,0134444302,Android How to Program,3,2017
9,0134289366,Android 6 for Programmers,3,2016


### author_ISBN

In [24]:
# View the final state of the author_ISBN table
pd.read_sql('SELECT * FROM author_ISBN', connection).head()

,id,isbn
0,1,0134289366
1,2,0134289366
2,5,0134289366
3,1,0135404673
4,2,0135404673


## CLOSE

In [25]:
# We no longer need to access the database, so disconnect from it
connection.close()